In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from transformers import ViTFeatureExtractor, ViTModel, ViTConfig, DistilBertModel, DistilBertConfig
from tqdm.notebook import tqdm
from torch.autograd import Variable
from datetime import datetime,timedelta
import time
from data_preparation import data_prepare

In [2]:
MODEL_PATH = './data/'

In [21]:
class PressureEncorder(nn.Module):
    def __init__(self, image_size = 41, patch_size = 4, num_channels = 40, encoder_stride = 4):
        super(PressureEncorder, self).__init__()
        config = ViTConfig(image_size = 41, patch_size = 4, num_channels = 10, encoder_stride = 4, num_attention_heads=4, hidden_size=40, num_hidden_layers=8)
        self.hidden_size = int((image_size // encoder_stride)**2 + 1) * config.hidden_size
        self.ViT = ViTModel(config)
        
        self.linear = nn.Linear(self.hidden_size + 20, 10)
        
    def forward(self, x):
        pressure, surge, time = x
        time = time.float()
        hidden = self.ViT(pressure).last_hidden_state.reshape(-1, self.hidden_size)
        x = torch.concat([hidden, surge, time], dim = 1)
        x = self.linear(x)
        return x

In [22]:
w = torch.linspace(1, 0.1, 10)[np.newaxis].to('cuda')

def custom_weighted_losses(output, target):
    loss = torch.mean(w * (output - target)**2)
    return loss

## Data Loading

Here the data is loaded and scaled

In [23]:
X_train = np.load('./data/X_train_surge_new.npz')
Y_train = pd.read_csv('./data/Y_train_surge.csv')
X_test = np.load('./data/X_test_surge_new.npz')

# train
slp_train = X_train['slp']
t_slp_train = X_train['t_slp']

t_surge1_input_train = X_train['t_surge1_input']
t_surge2_input_train = X_train['t_surge2_input']

surge1_input_train = X_train['surge1_input']
surge2_input_train = X_train['surge2_input']

mean_surge1_input_train = np.mean(surge1_input_train, axis=1)
std_surge1_input_train = np.std(surge1_input_train, axis=1)
mean_surge2_input_train = np.mean(surge2_input_train, axis=1)
std_surge2_input_train = np.std(surge2_input_train, axis=1)

scaled_surge1_input_train = (surge1_input_train - mean_surge1_input_train[:,None]) / std_surge1_input_train[:,None]
scaled_surge2_input_train = (surge2_input_train - mean_surge2_input_train[:,None]) / std_surge2_input_train[:,None]

t_surge1_output_train = X_train['t_surge1_output']
t_surge2_output_train = X_train['t_surge2_output']

# test
slp_test = X_test['slp']
t_slp_test = X_test['t_slp']

t_surge1_input_test = X_test['t_surge1_input']
t_surge2_input_test = X_test['t_surge2_input']

surge1_input_test = X_test['surge1_input']
surge2_input_test = X_test['surge2_input']

mean_surge1_input_test = np.mean(surge1_input_test, axis=1)
std_surge1_input_test = np.std(surge1_input_test, axis=1)
mean_surge2_input_test = np.mean(surge2_input_test, axis=1)
std_surge2_input_test = np.std(surge2_input_test, axis=1)

scaled_surge1_input_test = (surge1_input_test - mean_surge1_input_test[:,None]) / std_surge1_input_test[:,None]
scaled_surge2_input_test = (surge2_input_test - mean_surge2_input_test[:,None]) / std_surge2_input_test[:,None]

t_surge1_output_test = X_test['t_surge1_output']
t_surge2_output_test = X_test['t_surge2_output']

Now we need to divide the output

In [24]:
Y_1 = Y_train[['surge1_t0', 'surge1_t1', 'surge1_t2', 'surge1_t3', 'surge1_t4', 'surge1_t5', 'surge1_t6', 'surge1_t7', 'surge1_t8', 'surge1_t9']].to_numpy()
Y_2 = Y_train[['surge2_t0', 'surge2_t1', 'surge2_t2', 'surge2_t3', 'surge2_t4', 'surge2_t5', 'surge2_t6', 'surge2_t7', 'surge2_t8', 'surge2_t9']].to_numpy()

Code to have a series of pressures the same as the surges

In [25]:
train_dataloader1, val_dataloader1 = data_prepare(X_train)
train_dataloader2, val_dataloader2 = data_prepare(X_train, surge1 = False)
test_dataloader1 = data_prepare(X_test, train_set = False)
test_dataloader2 = data_prepare(X_test, train_set = False, surge1 = False)


## Data Training

Training of the transformer to compute the features from the pressure

surge_1

In [35]:
model1 = PressureEncorder()
device = torch.device('cuda')
model1 = model1.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=1e-4)
epochs = 70
batch_size=8
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)

In [36]:
epochs = 70
batch_size=8

for epoch in range(epochs):
    model1.train()
    for x1, x2, x3, y in tqdm(train_dataloader1, total = len(train_dataloader1), leave=False):
        x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        x3 = x3.type(torch.cuda.FloatTensor)
        y = y.type(torch.cuda.FloatTensor)
        optimizer.zero_grad()
        pred = model1((x1, x2, x3))
        loss = custom_weighted_losses(pred, y)
        loss.backward()
        optimizer.step()
        #loss = criterion(pred, y)
        #val_loss += loss.item()
        #p+=1
    scheduler.step()
    model1.eval()
    val_loss = 0
    with torch.no_grad():
        y_preds = []
        y_vali = []
        #p = 0
        for x1, x2, x3, y in tqdm(val_dataloader1, total = len(val_dataloader1), leave = False):
            x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
            x1 = x1.type(torch.cuda.FloatTensor)
            x2 = x2.type(torch.cuda.FloatTensor)
            x3 = x3.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.FloatTensor)
            pred = model1((x1, x2, x3))
            #loss = criterion(pred, y)
            #val_loss += loss.item()
            #p+=1
            y_preds.append(pred.to('cpu'))
            y_vali.append(y.to('cpu'))

    y_pred = []
    y_val = []
    for j in y_preds:
        for i in j:
            y_pred.append(i.numpy())
    y_pred = np.array(y_pred)
    for j in y_vali:
        for i in j:
            y_val.append(i.numpy())
    y_val = np.array(y_val)
        
    a=(y_pred-y_val)**2
    b = a*np.array([1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
    val_loss = b.mean()
    #val_loss =val_loss/p
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1: Validation Loss = 0.3334392530322404


  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 10: Validation Loss = 0.2831320359645592


  0%|          | 0/630 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [37]:
torch.save(model1.state_dict(), MODEL_PATH + 'model1.2')

In [ ]:
#model1 = PressureEncorder()
#model1.load_state_dict(torch.load(MODEL_PATH + 'model1'))
#model1.eval()
#model1.to(device)

In [43]:
model1.eval()
y_preds = []
with torch.no_grad():
    for x1, x2, x3 in tqdm(test_dataloader1, total = len(test_dataloader1), leave = False):
        x1, x2, x3 = x1.to(device), x2.to(device), x3.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        x3 = x3.type(torch.cuda.FloatTensor)
        pred = model1((x1, x2, x3))
        y_preds.append(pred.to(torch.device('cpu')))


  0%|          | 0/64 [00:00<?, ?it/s]

In [49]:
y_pred_1 = []
for j in y_preds:
    for i in j:
        y_pred_1.append(i.numpy())
y_pred_1 = np.array(y_pred_1)


Surge_2

In [50]:
model2 = PressureEncorder()
device = torch.device('cuda')
model2 = model2.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model2.parameters(), lr=1e-4)

In [51]:
epochs = 70
batch_size=8

for epoch in range(epochs):
    model2.train()
    for x1, x2, x3, y in tqdm(train_dataloader2, total = len(train_dataloader2), leave=False):
        x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        x3 = x3.type(torch.cuda.FloatTensor)
        y = y.type(torch.cuda.FloatTensor)
        optimizer.zero_grad()
        pred = model2((x1, x2, x3))
        loss = custom_weighted_losses(pred, y)
        loss.backward()
        optimizer.step()
        #loss = criterion(pred, y)
        #val_loss += loss.item()
        #p+=1
    model1.eval()
    val_loss = 0
    with torch.no_grad():
        y_preds = []
        y_vali = []
        #p = 0
        for x1, x2, x3, y in tqdm(val_dataloader2, total = len(val_dataloader2), leave = False):
            x1, x2, x3, y = x1.to(device), x2.to(device), x3.to(device), y.to(device)
            x1 = x1.type(torch.cuda.FloatTensor)
            x2 = x2.type(torch.cuda.FloatTensor)
            x3 = x3.type(torch.cuda.FloatTensor)
            y = y.type(torch.cuda.FloatTensor)
            pred = model2((x1, x2, x3))
            #loss = criterion(pred, y)
            #val_loss += loss.item()
            #p+=1
            y_preds.append(pred.to('cpu'))
            y_vali.append(y.to('cpu'))

    y_pred = []
    y_val = []
    for j in y_preds:
        for i in j:
            y_pred.append(i.numpy())
    y_pred = np.array(y_pred)
    for j in y_vali:
        for i in j:
            y_val.append(i.numpy())
    y_val = np.array(y_val)
        
    a=(y_pred-y_val)**2
    b = a*np.array([1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
    val_loss = b.mean()
    #val_loss =val_loss/p
    print(f'Epoch {epoch+1}: Validation Loss = {val_loss}')

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 1: Validation Loss = 0.3024353563492945


  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

...

  0%|          | 0/630 [00:00<?, ?it/s]

  0%|          | 0/70 [00:00<?, ?it/s]

Epoch 11: Validation Loss = 0.19915415261737024


  0%|          | 0/630 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [52]:
torch.save(model2.state_dict(), MODEL_PATH + 'model2.2')

In [10]:
#model2 = torch.load(MODEL_PATH + 'model2')

In [53]:
model2.eval()
y_preds_2 = []
with torch.no_grad():
    for x1, x2, x3 in tqdm(test_dataloader2, total = len(test_dataloader2), leave = False):
        x1, x2, x3 = x1.to(device), x2.to(device), x3.to(device)
        x1 = x1.type(torch.cuda.FloatTensor)
        x2 = x2.type(torch.cuda.FloatTensor)
        x3 = x3.type(torch.cuda.FloatTensor)
        pred = model2((x1, x2, x3))
        y_preds_2.append(pred.to(torch.device('cpu')))

  0%|          | 0/64 [00:00<?, ?it/s]

In [54]:
y_pred_2 = []
for j in y_preds_2:
    for i in j:
        y_pred_2.append(i.numpy())
y_pred_2 = np.array(y_pred_2)

In [55]:
y_pred_2.shape

(509, 10)

In [58]:
y_test_1v1cities_1 = np.concatenate([y_pred_1, y_pred_2], axis=1)

In [59]:
y_columns = [f'surge1_t{i}' for i in range(10)] + [f'surge2_t{i}' for i in range(10)]
y_test_benchmark = pd.DataFrame(data=y_test_1v1cities_1, columns=y_columns, index=X_test['id_sequence'])
y_test_benchmark.to_csv('y_test_1v1cities_1.csv', index_label='id_sequence', sep=',')

In [60]:
y_test_1v1cities_1.shape

(509, 20)